# Quickstart

In this quickstart guide, you'll get up and running with a simple agent that can
search the web using [Tavily Search API](https://tavily.com/). The code is fully
configurable, meaning you can swap out components, customize the execution flow,
and extend to add any custom code or tooling.

First install the required dependencies:

```bash
npm install @langchain/langgraph @langchain/openai @langchain/community
```

Then set the required environment variables. Optionally, set up
[LangSmith](https://docs.smith.langchain.com/) for best-in-class observability:

In [1]:
// process.env.OPENAI_API_KEY = "sk_...";
// process.env.TAVILY_API_KEY = "sk_...";

// Optional, add tracing in LangSmith
process.env.LANGCHAIN_API_KEY = "ls__...";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "Quickstart: LangGraphJS";


Quickstart: LangGraphJS


In [2]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";
import { ChatOpenAI } from "@langchain/openai";
import { MemorySaver } from "@langchain/langgraph";
import { HumanMessage } from "@langchain/core/messages";
import { createReactAgent } from "@langchain/langgraph/prebuilt";

// Define the tools for the agent to use
const agentTools = [new TavilySearchResults({ maxResults: 1 })];
const agentModel = new ChatOpenAI({ temperature: 0 });

// Initialize memory to persist state between graph runs
const agentCheckpointer = new MemorySaver();
const agent = createReactAgent({
  llm: agentModel,
  tools: agentTools,
  checkpointSaver: agentCheckpointer,
});

// Now it's time to use!
const agentFinalState = await agent.invoke(
  { messages: [new HumanMessage("what is the weather in sf")] },
  { configurable: { thread_id: "42" } },
);

console.log(
  agentFinalState.messages[agentFinalState.messages.length - 1].content,
);

const agentNextState = await agent.invoke(
  { messages: [new HumanMessage("what about ny")] },
  { configurable: { thread_id: "42" } },
);

console.log(
  agentNextState.messages[agentNextState.messages.length - 1].content,
);

The current weather in San Francisco is as follows:
- Temperature: 82.9°F (28.3°C)
- Condition: Sunny
- Wind: 20.2 km/h from the NW
- Humidity: 41%
- Visibility: 9.0 miles
- UV Index: 5.0

For more details, you can visit [Weather in San Francisco](https://www.weatherapi.com/).
The current weather in New York is as follows:
- Temperature: 72.0°F (22.2°C)
- Condition: Partly cloudy
- Wind: 3.6 km/h from the N
- Humidity: 57%
- Cloud Cover: 75%
- UV Index: 5.0

For more details, you can visit [Weather in New York](https://www.weatherapi.com/).


## How does it work?

The
[createReactAgent](https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html)
constructor lets you create a simple tool-using LangGraph agent in a single line
of code. This can be powerful, but what makes LangGraph powerful is that you can
fully decompose the agent for finer-grained control over its behavior. The
following code creates an agent with the same behavior as the example above, but
you can clearly see the execution logic and how you could customize it.

In [ ]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";
import { ChatOpenAI } from "@langchain/openai";
import { MemorySaver } from "@langchain/langgraph";
import { HumanMessage } from "@langchain/core/messages";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { END, START, StateGraph, StateGraphArgs } from "@langchain/langgraph";

// Define the state interface
interface AgentState {
  messages: HumanMessage[];
}

// Define the graph state
const graphState: StateGraphArgs<AgentState>["channels"] = {
  messages: {
    value: (x: HumanMessage[], y: HumanMessage[]) => x.concat(y),
    default: () => [],
  },
};

// Define the tools for the agent to use
const tools = [new TavilySearchResults({ maxResults: 1 })];

const toolNode = new ToolNode<AgentState>(tools);

const model = new ChatOpenAI({ temperature: 0 }).bindTools(tools);

// Define the function that determines whether to continue or not
function shouldContinue(state: AgentState): "tools" | typeof END {
  const messages = state.messages;

  const lastMessage = messages[messages.length - 1];

  // If the LLM makes a tool call, then we route to the "tools" node
  if (lastMessage.additional_kwargs.tool_calls) {
    return "tools";
  }
  // Otherwise, we stop (reply to the user)
  return END;
}

// Define the function that calls the model
async function callModel(state: AgentState) {
  const messages = state.messages;

  const response = await model.invoke(messages);

  // We return a list, because this will get added to the existing list
  return { messages: [response] };
}

// Define a new graph
const workflow = new StateGraph<AgentState>({ channels: graphState })
  .addNode("agent", callModel)
  .addNode("tools", toolNode)
  .addEdge(START, "agent")
  .addConditionalEdges("agent", shouldContinue)
  .addEdge("tools", "agent");

// Initialize memory to persist state between graph runs
const checkpointer = new MemorySaver();

// Finally, we compile it!
// This compiles it into a LangChain Runnable.
// Note that we're (optionally) passing the memory when compiling the graph
const app = workflow.compile({ checkpointer });

// Use the agent
const finalState = await app.invoke(
  { messages: [new HumanMessage("what is the weather in sf")] },
  { configurable: { thread_id: "42" } },
);

console.log(finalState.messages[finalState.messages.length - 1].content);

const nextState = await app.invoke(
  { messages: [new HumanMessage("what about ny")] },
  { configurable: { thread_id: "42" } },
);

console.log(nextState.messages[nextState.messages.length - 1].content);

## Next Steps

Now that you've created a custom agent, check out the
[tutorials](https://langchain-ai.github.io/langgraphjs/tutorials/) to learn
LangGraph by implementing different types of end-to-end workflows from RAG to
multi-agent swarms, or consult the
[how-to guides](https://langchain-ai.github.io/langgraphjs/how-tos/) for more
examples of how to implement different design patterns.